A notebook to test a DPT model.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
import os
# import numpy as np
from natsort import natsorted

from train_dpt import DPTSolver
from notebooks.utils import *

Download a model from a checkpoint. The last checkpoint is used.

In [ ]:
def get_checkpoint(run_name):
    root_dir = os.path.join("../../results", "DPT_3", run_name, "checkpoints")
    checkpoint = natsorted(os.listdir(root_dir))[-1]
    checkpoint_file = os.path.join(root_dir, checkpoint)
    return checkpoint_file

In [ ]:
device = "cuda:0"
run = "y9t9d0jr"
checkpoint_file = get_checkpoint(run)
model = DPTSolver.load_from_checkpoint(checkpoint_file)
model = model.to(device)

Define a budget.

In [ ]:
budget = 2 * model.config["model_params"]["seq_len"]

Run the model in an online inference mode for a given problem.

In [ ]:
read_dir = "../data/normal"
problem = "Normal(0, 1)"
logs = run_model(model, read_dir, problem, name="AD", budget=budget)

Here one can go to `utils.py` file and adjust the printing settings. Namely, it is possible to move the legend or turn on clipping.

In [ ]:
show_meta_results({problem: logs})

Plot the results along with other solvers results to compare. 

In [ ]:
read_dir = "../results/normal"
problem_list = (problem,) # natsorted(os.listdir(read_dir))
solver_list = ('RandomSearch', 'PSO', 'PROTES')

meta_results = defaultdict(dict)
for problem in problem_list:
    for solver in solver_list:
        meta_results[problem][solver] = get_meta_results(problem, solver, read_dir, suffix='test', budget=budget)

In [ ]:
meta_results[problem] |= logs
show_meta_results(meta_results)

Stuff

In [ ]:
# warmup = 0
# model.config["temperature"] = lambda x: math.sqrt(x)
# model.config["temperature"] = lambda x: 5 - 4 * x
# model.config["temperature"] = lambda x: 1 / math.sqrt(1 + x)

In [ ]:
# for k, v in logs.items():
#     print(k, len(v["m_list"]), len(v["y_list (mean)"]))